In [1]:
import os
import csv
from supabase import create_client, Client
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [2]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            print(row)
            if row != {'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}:
                game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [14]:
round = "1"
update_game_schedule(f"2024/game-schedule-round-{round}-2024-ncaa-tournament.csv")

{'game_date': '2024-03-19', 'team_2_id': 'wagner', 'team_1_id': 'howard', 'game_time': '19:00', 'round_num': '1', 'competition_id': '5'}
{'game_date': '2024-03-19', 'team_2_id': 'colorado-state', 'team_1_id': 'virginia', 'game_time': '21:10', 'round_num': '1', 'competition_id': '5'}
{'game_date': '2024-03-20', 'team_2_id': 'grambling', 'team_1_id': 'montana-state', 'game_time': '19:00', 'round_num': '1', 'competition_id': '5'}
{'game_date': '2024-03-20', 'team_2_id': 'colorado', 'team_1_id': 'boise-state', 'game_time': '21:10', 'round_num': '1', 'competition_id': '5'}
{'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}
{'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}
{'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}
{'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id

In [11]:
import pandas as pd
import datetime
def generate_game_scoring_sheet(competition_id: int = 1, date = datetime.date.today()):
    data = supabase.table("players_in_games_view").select("*").eq("game_date", date).eq('competition_id', competition_id).execute()
    print(data)
    df = pd.DataFrame(data.data)
    df['points'] = None
    df['lost'] = None
    df['inactive'] = None
    # df['rebounds'] = 0
    # df['assists'] = 0
    df.to_csv(f"2024/{date}-game-scoring-2024-ncaa-tournament.csv", index=False, columns=['game_time', 'team_unique', 'lost', 'player_unique', 'points','inactive','game_id'])
    # 'rebounds', 'assists'])

In [16]:
# date = datetime.date.today()
date = "2024-03-20"
generate_game_scoring_sheet(5, date=date)

data=[{'player_unique': 'kintavious-dozier-1', 'team_unique': 'grambling', 'game_id': 169, 'game_date': '2024-03-20', 'game_time': '19:00:00', 'round_num': 1, 'competition_id': 5}, {'player_unique': 'tramichael-moton-1', 'team_unique': 'grambling', 'game_id': 169, 'game_date': '2024-03-20', 'game_time': '19:00:00', 'round_num': 1, 'competition_id': 5}, {'player_unique': 'jourdan-smith-1', 'team_unique': 'grambling', 'game_id': 169, 'game_date': '2024-03-20', 'game_time': '19:00:00', 'round_num': 1, 'competition_id': 5}, {'player_unique': 'jalen-johnson-23', 'team_unique': 'grambling', 'game_id': 169, 'game_date': '2024-03-20', 'game_time': '19:00:00', 'round_num': 1, 'competition_id': 5}, {'player_unique': 'antwan-burnett-1', 'team_unique': 'grambling', 'game_id': 169, 'game_date': '2024-03-20', 'game_time': '19:00:00', 'round_num': 1, 'competition_id': 5}, {'player_unique': 'terrence-lewis-2', 'team_unique': 'grambling', 'game_id': 169, 'game_date': '2024-03-20', 'game_time': '19:00:0

In [22]:
def update_scores_from_csv(current_round: int, competition_id: int, game_scoring_csv = f"{datetime.date.today()}-game-scoring-2023-ncaa-tournament.csv"):
    player_game_insert = []
    losing_teams = []
    inactive_players = []
    team_1 = ""
    team_2 = ""
    with open(game_scoring_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            if row['lost'].upper() == 'L':
                losing_teams.append(row['team_unique'])
            if row['inactive'].upper() == 'I':
                inactive_players.append(row['player_unique'])
            if row['points'] != '':
                team_2 = team_1
                team_1 = row['team_unique']
                player_game_row = {
                    "player_unique": row['player_unique'],
                    "game_id": row['game_id'],
                    "points": row['points'],
                }
                player_game_insert.append(player_game_row)

    data = supabase.table("player_game").upsert(player_game_insert,
                                         ignore_duplicates=False, on_conflict="game_id, player_unique").execute()
    updated = supabase.table("competition_updated").insert({"competition_id": competition_id, "current_round": current_round}).execute()
    for team in losing_teams:
        loser_data = supabase.table('team_competition').update({'round_eliminated': current_round}).eq('team_unique', team).eq('competition_id', competition_id).execute()
        print("losers")
        print(loser_data)
    for player in inactive_players:
        player_data = supabase.table('player_competition').update({'inactive': True}).eq('player_unique', player).eq('competition_id', competition_id).execute()
        print("inactive players")
        print(player_data)
    print(data)

In [33]:
# date = datetime.date.today()
current_round=2
date = "2024-03-21"
update_scores_from_csv(current_round=current_round, competition_id=5, game_scoring_csv = f"2024/{date}-game-scoring-2024-ncaa-tournament.csv")


losers
data=[{'team_unique': 'mississippi-state', 'competition_id': 5, 'league_unique': 'ncaambb', 'seed': 8, 'overall_seed': 32, 'region': 'WEST', 'round_eliminated': 2, 'round_started': 2, 'team_stats': {'pace': 68.9, 'conference': 'sec', 'offensive_rating': 108.1, 'assist_percentage': 53.3, 'turnover_percentage': 15.9, 'strength_of_schedule': 9.22, 'free_throw_attempt_rate': 0.379, 'three_point_attempt_rate': 0.382, 'true_shooting_percentage': 0.546, 'effective_field_goal_percentage': 0.517, 'two_point_field_goal_percentage': 0.535, 'three_point_field_goal_percentage': 0.325, 'opp_effective_field_goal_percentage': 0.477}, 'stats_thru': '2024-03-17', 'team_win_loss': {'wins': 21.0, 'losses': 13.0, 'games_played': 0, 'conference_wins': 8.0, 'conference_losses': 10.0}}] count=None
losers
data=[{'team_unique': 'brigham-young', 'competition_id': 5, 'league_unique': 'ncaambb', 'seed': 6, 'overall_seed': 17, 'region': 'EAST', 'round_eliminated': 2, 'round_started': 2, 'team_stats': {'pace'